# Instalações das bibliotecas

In [ ]:
!pip install pytelegrambotapi
!pip install pyUFbr
!pip install unidecode
!pip install spacy
!python -m spacy download pt_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 841.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyUFbr: filename=pyUFbr-0.1.0-py3-none-any.whl size=63704 sha256=bd491b6d73cbe2ed364b45d57901e21c4ed67e5f391ffe8b4f49efc7994632f1
  Stored in directory: /root/.cache/pip/wheels/b0/5e/a6/df5e053e9847cb01a3031696952ea32a5350c95635bb1d89e3
Successfully built pyUFbr
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 65.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this

# Chatbot

In [ ]:
import telebot
import unidecode
from pyUFbr.baseuf import ufbr
import re
import spacy
nlp = spacy.load('pt_core_news_sm')

API_TOKEN = '7465156973:AAEUyiaHXYrwqtpoDLjA9_3EPTN2kFR6k70'  # Substitua pelo seu token do Telegram
bot = telebot.TeleBot(API_TOKEN)

user_data = {}

def gerar_link_imovel(tipo, estado, cidade, valor_minimo, valor_maximo, tipo_imovel, quartos, banheiro, area_minima, area_maxima, mobiliado, aceita_pet):
    valor_minimo_formatado = valor_minimo.replace("R$", "").replace(".", "").strip()
    valor_maximo_formatado = valor_maximo.replace("R$", "").replace(".", "").strip()

    link = (f"https://www.quintoandar.com.br/{tipo}/imovel/{cidade.lower().replace(' ', '-')}-{estado.lower()}"
            f"/de-{valor_minimo_formatado}-a-{valor_maximo_formatado}-reais/{tipo_imovel}/{quartos}-quartos/{banheiro}-banheiros/de-{area_minima}-a-{area_maxima}-m2/{mobiliado}/{aceita_pet}")
    return link

@bot.message_handler(commands=['imovel'])
def iniciar_busca_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id] = {}
    texto = 'Olá, seja bem vindo à imobiliária de Ana e Erika! Você gostaria de alugar ou comprar um imóvel?'
    bot.send_message(chat_id, texto)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'tipo' not in user_data[message.chat.id])
def confirmar_tipo(message):
    chat_id = message.chat.id
    doc = nlp(message.text.lower())
    user_data[chat_id]['tipo'] = None

    tipo_keywords = {'comprar': ['comprar', 'adquirir', 'compra'], 'alugar': ['alugar', 'locar', 'aluguel']}
    for token in doc:
        lemma = token.lemma_
        for tipo, keywords in tipo_keywords.items():
            if lemma in keywords:
                user_data[chat_id]['tipo'] = tipo
                break

    if user_data[chat_id]['tipo'] is None:
        texto = "Desculpe, não entendi. Você gostaria de alugar ou comprar um imóvel?"
        bot.send_message(chat_id, texto)
    else:
        perguntar_estado_cidade(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'estado_cidade' not in user_data[message.chat.id])
def perguntar_estado_cidade(message):
    chat_id = message.chat.id
    user_data[message.chat.id]['estado_cidade'] = 'aguardando'
    texto = 'Em qual estado e cidade? Por favor, informe a sigla do estado (UF) e a cidade'
    bot.send_message(chat_id, texto)

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('estado_cidade') == 'aguardando')
def validar_estado_cidade(message):
    chat_id = message.chat.id
    texto_normalizado = unidecode.unidecode(message.text.lower())

    estado_encontrado, cidade_encontrada = None, None
    for estado in ufbr.list_uf:
        if estado.lower() in texto_normalizado:
            estado_encontrado = estado
            cidades = ufbr.list_cidades(estado)
            for cidade in cidades:
                if unidecode.unidecode(cidade.lower()) in texto_normalizado:
                    cidade_encontrada = cidade
                    break
            if cidade_encontrada:
                break

    if not estado_encontrado or not cidade_encontrada:
        bot.send_message(chat_id, "Desculpe, não consegui entender. Por favor, informe a sigla do estado (UF) e a cidade.")
    else:
        user_data[chat_id]['estado_cidade'] = (estado_encontrado, cidade_encontrada)
        perguntar_valor_minimo(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'valor_minimo' not in user_data[message.chat.id])
def perguntar_valor_minimo(message):
    chat_id = message.chat.id
    user_data[chat_id]['valor_minimo'] = '0'
    bot.send_message(chat_id, "Qual o valor mínimo?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('valor_minimo') == '0')
def confirmar_valor_minimo(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número mínimo.")
    else:
        user_data[chat_id]['valor_minimo'] = numeros[0]
        perguntar_valor_maximo(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'valor_maximo' not in user_data[message.chat.id])
def perguntar_valor_maximo(message):
    chat_id = message.chat.id
    user_data[chat_id]['valor_maximo'] = '0'
    bot.send_message(chat_id, "Qual o valor máximo?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('valor_maximo') == '0')
def confirmar_valor_maximo(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número máximo.")
    else:
        user_data[chat_id]['valor_maximo'] = numeros[0]
        perguntar_tipo_imovel(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'tipo_imovel' not in user_data[message.chat.id])
def perguntar_tipo_imovel(message):
    chat_id = message.chat.id
    user_data[chat_id]['tipo_imovel'] = "aguardando"
    bot.send_message(chat_id, "É apartamento ou casa?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('tipo_imovel') == 'aguardando')
def confirmar_tipo_imovel(message):
    chat_id = message.chat.id
    doc = nlp(message.text.lower())
    apartamento_keywords = ['apartamento', 'ape', 'apê', 'flat', 'apart']
    casa_keywords = ['casa', 'sobrado', 'residencia', 'moradia']

    for token in doc:
        lemma = token.lemma_
        if lemma in apartamento_keywords:
            user_data[chat_id]['tipo_imovel'] = 'apartamento'
            break
        elif lemma in casa_keywords:
            user_data[chat_id]['tipo_imovel'] = 'casa'
            break

    if user_data[chat_id]['tipo_imovel'] is None:
        bot.send_message(chat_id, "Desculpe, não entendi. Você está procurando um apartamento ou uma casa?")
    else:
        perguntar_quartos(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'quartos' not in user_data[message.chat.id])
def perguntar_quartos(message):
    chat_id = message.chat.id
    user_data[chat_id]['quartos'] = "0"
    bot.send_message(chat_id, "Quantos quartos?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('quartos') == '0')
def confirmar_quartos(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número.")
    else:
        user_data[chat_id]['quartos'] = numeros[0]
        perguntar_banheiro(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'banheiro' not in user_data[message.chat.id])
def perguntar_banheiro(message):
    chat_id = message.chat.id
    user_data[chat_id]['banheiro'] = "0"
    bot.send_message(chat_id, "Quantos banheiros?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('banheiro') == '0')
def confirmar_banheiro(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número.")
    elif int(numeros[0]) < 1 or int(numeros[0]) > 4:
        bot.send_message(chat_id, "Por favor, informe um número entre 1 e 4.")
    else:
        user_data[chat_id]['banheiro'] = numeros[0]
        perguntar_area_minima(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'area_minima' not in user_data[message.chat.id])
def perguntar_area_minima(message):
    chat_id = message.chat.id
    user_data[chat_id]['area_minima'] = "0"
    bot.send_message(chat_id, "Qual a área mínima em metros quadrados?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('area_minima') == '0')
def confirmar_area_minima(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número mínimo.")
    else:
        user_data[chat_id]['area_minima'] = numeros[0]
        perguntar_area_maxima(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'area_maxima' not in user_data[message.chat.id])
def perguntar_area_maxima(message):
    chat_id = message.chat.id
    user_data[chat_id]['area_maxima'] = "0"
    bot.send_message(chat_id, "Qual a área máxima em metros quadrados?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('area_maxima') == '0')
def confirmar_area_maxima(message):
    chat_id = message.chat.id
    numeros = re.findall(r'\d+', message.text)
    if len(numeros) != 1:
        bot.send_message(chat_id, "Por favor, informe apenas um número máximo.")
    else:
        user_data[chat_id]['area_maxima'] = numeros[0]
        perguntar_mobilia(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'mobiliado' not in user_data[message.chat.id])
def perguntar_mobilia(message):
    chat_id = message.chat.id
    user_data[chat_id]['mobiliado'] = "0"
    bot.send_message(chat_id, "Você quer que esteja mobiliado?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('mobiliado') == '0')
def confirmar_mobilia(message):
    chat_id = message.chat.id
    doc = nlp(message.text.lower())
    affirmative_responses = ['sim', 'claro', 'com certeza', 'quero', 'desejo', 'affirmativo']
    negative_responses = ['não', 'nao', 'nunca', 'dispenso', 'não quero', 'negativo']

    for token in doc:
        lemma = token.lemma_
        if lemma in affirmative_responses:
            user_data[chat_id]['mobiliado'] = 'mobiliado'
            break
        elif lemma in negative_responses:
            user_data[chat_id]['mobiliado'] = 'nao-mobiliado'
            break

    if user_data[chat_id]['mobiliado'] is None:
        bot.send_message(chat_id, "Desculpe, não entendi. Você gostaria que o imóvel estivesse mobiliado?")
    else:
        perguntar_aceita_pet(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'aceita_pet' not in user_data[message.chat.id])
def perguntar_aceita_pet(message):
    chat_id = message.chat.id
    user_data[chat_id]['aceita_pet'] = "0"
    bot.send_message(chat_id, "Você quer que aceite pet?")

@bot.message_handler(func=lambda message: message.chat.id in user_data and user_data[message.chat.id].get('aceita_pet') == '0')
def confirmar_aceita_pet(message):
    chat_id = message.chat.id
    doc = nlp(message.text.lower())
    affirmative_responses = ['sim', 'claro', 'com certeza', 'quero', 'desejo', 'affirmativo']
    negative_responses = ['não', 'nao', 'nunca', 'dispenso', 'não quero', 'negativo']

    for token in doc:
        lemma = token.lemma_
        if lemma in affirmative_responses:
            user_data[chat_id]['aceita_pet'] = 'aceita-pets'
            break
        elif lemma in negative_responses:
            user_data[chat_id]['aceita_pet'] = 'nao-aceita-pets'
            break

    if user_data[chat_id]['aceita_pet'] is None:
        bot.send_message(chat_id, "Desculpe, não entendi. Você gostaria que o imóvel aceitasse pets?")
    else:
        gerar_link_final(message)

@bot.message_handler(func=lambda message: message.chat.id in user_data and 'banheiro' in user_data[message.chat.id])
def gerar_link_final(message):
    chat_id = message.chat.id
    required_keys = ['tipo', 'estado_cidade', 'valor_minimo', 'valor_maximo', 'tipo_imovel', 'quartos', 'banheiro', 'area_minima', 'area_maxima', 'mobiliado', 'aceita_pet']
    missing_keys = [key for key in required_keys if key not in user_data[chat_id]]

    if missing_keys:
        bot.send_message(chat_id, f"Informação faltando: {', '.join(missing_keys)}. Por favor, responda às perguntas corretamente.")
        return

    tipo = user_data[chat_id]['tipo']
    estado, cidade = user_data[chat_id]['estado_cidade']
    valor_minimo = user_data[chat_id]['valor_minimo']
    valor_maximo = user_data[chat_id]['valor_maximo']
    tipo_imovel = user_data[chat_id]['tipo_imovel']
    quartos = user_data[chat_id]['quartos']
    banheiro = user_data[chat_id]['banheiro']
    area_minima = user_data[chat_id]['area_minima']
    area_maxima = user_data[chat_id]['area_maxima']
    mobiliado = user_data[chat_id]['mobiliado']
    aceita_pet = user_data[chat_id]['aceita_pet']

    link_imovel = gerar_link_imovel(tipo, estado, cidade, valor_minimo, valor_maximo, tipo_imovel, quartos, banheiro, area_minima, area_maxima, mobiliado, aceita_pet)
    bot.send_message(chat_id, f"Aqui está o link para imóveis em {cidade}:\n{link_imovel}")
    del user_data[chat_id]

Ativação do Chatbot

In [ ]:
bot.polling()